# TF Record file generator

In this notebook we generate the .record file, that contains the dataset for traffic light classification training. We do this generation in order to use the output file with TF object detection API.

### Import Libraries

In [1]:
import tensorflow as tf
import yaml
import os
from object_detection.utils import dataset_util

In [2]:
output_path_train = './data/dataset_training.record'
output_path_eval = './data/dataset_eval.record'

LABEL_DICT =  {
    "Green" : 1,
    "Red" : 2,
    "Yellow" : 3,
    "off" : 4,
    }

In [3]:
def create_tf_example(example):
    
    # Udacity real data set
    height = 1096 # Image height
    width = 1368 # Image width

    filename = example['filename'] # Filename of the image. Empty if image is not from file
    filename = filename.encode()

    with tf.io.gfile.GFile(example['filename'], 'rb') as fid:
        encoded_image = fid.read()

    image_format = 'jpg'.encode() 

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
                # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
                # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    for box in example['annotations']:
        #print("adding box")
        xmins.append(float(box['xmin'] / width))
        xmaxs.append(float((box['xmin'] + box['x_width']) / width))
        ymins.append(float(box['ymin'] / height))
        ymaxs.append(float((box['ymin']+ box['y_height']) / height))
        classes_text.append(box['class'].encode())
        classes.append(int(LABEL_DICT[box['class']]))


    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example


In [4]:
import random

def main(_):
    VALIDATION = 0.1
    
    INPUT_YAML = "data/dataset_orig/real_data_annotations.yaml"
    examples = yaml.load(open(INPUT_YAML, 'rb').read())

    #examples = examples[:10]  # for testing
    len_examples = len(examples)
    print("Loaded ", len(examples), "examples")

    for i in range(len(examples)):
        examples[i]['filename'] = os.path.abspath(os.path.join(os.path.dirname(INPUT_YAML), examples[i]['filename']))
        
    random.shuffle(examples)

    print(int(VALIDATION*len(examples)))
    writer = tf.io.TFRecordWriter(output_path_train)
    counter = 0
    for example in examples[:-int(VALIDATION*len(examples))]:
        tf_example = create_tf_example(example)
        writer.write(tf_example.SerializeToString())

        if counter % 10 == 0:
            print("Percent done", (counter/len_examples)*100)
        counter += 1

    print("Total counter images training: %d "%counter)
    
    writer = tf.io.TFRecordWriter(output_path_eval)
    counter = 0
    for example in examples[-int(VALIDATION*len(examples))::]:
        tf_example = create_tf_example(example)
        writer.write(tf_example.SerializeToString())

        if counter % 10 == 0:
            print("Percent done", (counter/len_examples)*100)
        counter += 1

    print("Total counter images eval: %d "%counter)
    writer.close()

In [5]:
tf.compat.v1.app.run()

/home/lborgnino/miniconda3/envs/carnd-advdl-odlab/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


Loaded  159 examples
15
Percent done 0.0
Percent done 6.289308176100629
Percent done 12.578616352201259
Percent done 18.867924528301888
Percent done 25.157232704402517
Percent done 31.446540880503143
Percent done 37.735849056603776
Percent done 44.0251572327044
Percent done 50.314465408805034
Percent done 56.60377358490566
Percent done 62.893081761006286
Percent done 69.18238993710692
Percent done 75.47169811320755
Percent done 81.76100628930818
Percent done 88.0503144654088
Total counter images training: 144 
Percent done 0.0
Percent done 6.289308176100629
Total counter images eval: 15 


SystemExit: 

/home/lborgnino/miniconda3/envs/carnd-advdl-odlab/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
